In [1]:
import sys
import os
import ssm
import itertools

import autograd.numpy          as np
import pandas                  as pd

import matplotlib.pyplot       as plt
import seaborn                 as sns

from   sklearn.model_selection import KFold
from   sklearn.mixture         import GaussianMixture

sns.set_style("white")
sns.set_context("talk")

## Autoregressive Hidden Markov Models for modelling baby movement
Using the [ssm](https://github.com/lindermanlab/ssm) package.  

### Aims
- Fit (AR)HMM models to pre-processed movement data  
- Compare model fit when including autoregressive terms in observations  
- Use cross-validation to determine number of hidden states and lag

Using Hidden Markov Models (HMM), a continuous, $D$-dimension $\times$ $t$-timestep timeseries, $X$, can be modelled as a Markov process, $Z$, with a set of $K$ hidden states, {$z_1$, $z_2$, $...$, $z_k$}, that are not directly observable. Each state is associated with a $D$-dimensional mean $\mu_k$ and $D \times D$ covariance matrix $S_k$ from which the multivariate observations are drawn at each time step. The hidden states are assumed to progress as a Markov process, where the the next state is dependent only on the current state: $p(z_t | z_{t-1})$. Switches between states are governed by a $K \times K$ transition matrix, $T$ containing state transition probabilities, $\phi$, where $\phi_{zz'}$ indicates the probability of transitioning from state $z$ at time $t$ to state $z'$ at time $t+1$. As any given state is only dependent on the previous state, HMMs do not capture longer-term correlations in timeseries data.

In an AutoRegressive HMM (ARHMM), the observations, $x_t$ at time $t$ are dependent on $both$ the hidden state, $z_t$, and the observations at previous timespoints, {$x_{t-1}$, $x_{t-2}$, $...$, $x_{t-n}$} with $n$ determined by the degree of lag specified. As such the observation model for a given state, $z_k$, is defined as:

$$ x_t | x_{t-1:L}, z_k \sim N(\sum_{l=1}^LA_k^{(l)}x_{t-l}+b_k,S_k)$$   

.... i think this is the right formulation for multiple lags...

where $x_t$ is a set of observations at time $t$, $A_k^{(l)}$ is a matrix containing the linear dynamics for the given state (the relationship between observations at time $t$ and at a given lag, $l$), $b_k$ is a state-dependent bias (average value of each variable in each state) and $S_k$ is a state-dependent diagonal covariance function where $S_{i}=0$ when $i \neq j$, modelling observation noise.

<img src="graphical_model.png" width="500"/>




### Parameters

In [2]:
CV = 5 # number of cross-validation folds

# ARHMM
ARHMM_OBS = ['diagonal_robust_ar']
ARHMM_K = [1, 2, 5, 10, 15, 25] #k=1 is standard AR
ARHMM_LAGS = [1, 2, 5, 10, 15]
ARHMM_PARAMS = list(itertools.product(ARHMM_OBS, ARHMM_K, ARHMM_LAGS))

# HMM (no autoregression)
HMM_OBS = ['diagonal_gaussian']
HMM_K = [1, 2, 5, 10, 15, 25] 
HMM_PARAMS = list(itertools.product(HMM_OBS, HMM_K))

ALL_HMM_PARAMS = HMM_PARAMS + ARHMM_PARAMS

# STICKY ARHMM
ARHMM_K = [5, 10, 15, 25] 
ARHMM_LAGS = [2, 5, 10]
ARHMM_KAPPA = [100]
ARHMM_TRANS = ['sticky']

STICKY_ARHMM_PARAMS = list(itertools.product(ARHMM_OBS, ARHMM_K, ARHMM_LAGS, ARHMM_TRANS, ARHMM_KAPPA))

# GMM (no state progression)
GMM_OBS = ['diag']
GMM_K = [1, 2, 5, 10, 15, 25]

GMM_PARAMS = list(itertools.product(GMM_OBS, GMM_K))

### Load preprocessed data

Data represents a $subject \times feature \times timepoint$ array. Where $features$ are a combination of postural and dynamic features extracted from keypoint trajectory data. Subect data are split into cross-validation folds, ensuring no subject has video in both test and train in any given fold.

In [3]:
data = np.load('outputs/processed_timeseries_data.npy')
info = pd.read_csv('outputs/all_subject_info.csv')

# take top half
#data = data[:150]
#info = info.iloc[:150]

print('data shape: ', np.shape(data))
num_vids, num_features, num_timepoints = data.shape


#train test 
# split on participant (not video) to ensure that same subject data are not in both train and test
unique_participants = info.drop_duplicates(subset = 'idnum', keep = 'first')
X = unique_participants['idnum'].values
# get splits
kf = KFold(n_splits=CV, shuffle=True, random_state=1001).split(X)
# get participant train and test ids for each fold
kfold_idx = [(X[train_idx], X[test_idx]) for train_idx, test_idx in kf]
# get index in 'info' and 'data' for entries that match the unique participant id
kfold_idx = [(np.where(info['idnum'].isin(trainidx))[0], np.where(info['idnum'].isin(testidx))[0])  for trainidx, testidx in kfold_idx]
# check there are no participants in both train and test in a given fold
for k in np.arange(CV):
    assert len(set(info.iloc[kfold_idx[k][0]].participant.unique()) & set(info.iloc[kfold_idx[k][1]].participant.unique())) == 0

data shape:  (486, 38, 4500)


### Run cross-validation
HMMs for each parameter set are run in a k-fold cross-valisation with per-observation average log likelihood in both train and test data as a measure of model fit. The score reflect the likelihood that a given $D$-dimensional observation could have been generated by a given model. We compare ARHMM at different lags and with different states with standard HMM (no autocorrelation) over a number of states and Gaussian Mixture Models (stationary model where observations are simply drawn from separate distributions and we do not model state progression)

In [4]:
# space for results
hmm_results = np.zeros((len(ALL_HMM_PARAMS), 4, CV)) # model x train/test scores x folds
shmm_results = np.zeros((len(STICKY_ARHMM_PARAMS), 4, CV)) # model x train/test scores x folds
gmm_results = np.zeros((len(GMM_PARAMS), 4, CV)) # model x train/test scores x folds

for f, (train_idx, test_idx) in enumerate(kfold_idx):
    print('FOLD {:}----------------------------------------------'.format(f+1))
    train_data = [data[i,:,:].T for i in train_idx]
    test_data = [data[i,:,:].T for i in test_idx]
    
    # scale to unit SD prior to modelling using train data to estimate
    obs_sd = np.std(np.concatenate(train_data), 0)
    train_data_scaled = [i / obs_sd for i in train_data]
    test_data_scaled = [i / obs_sd for i in test_data]

    # parse model parameters
    for n_p, p_set in enumerate(ALL_HMM_PARAMS):
        if len(p_set)==3:
            obs, k, lag = [*p_set]
            model = ssm.HMM(k, num_features, observations=obs, observation_kwargs={'lags':lag})
            print('fitting {:} model with k={:} and lags={:}'.format(obs, k, lag))
        else:
            assert(len(p_set)==2)
            obs, k = [*p_set]
            lag = None
            model = ssm.HMM(k, num_features, observations=obs)
            print('fitting {:} model with k={:}'.format(obs, k))

        # fit
        # shuffle data before fitting
        ridx = np.random.choice(np.arange(len(train_data_scaled)), size=len(train_data_scaled), replace=False)
        train_data_shuffled = [train_data_scaled[i] for i in ridx]
        # model fit
        lls = model.fit(train_data_shuffled, method='em', num_iters=50)

        # record scores
        train_score = model.log_likelihood(train_data_scaled)
        train_av_score = train_score / (len(train_data_scaled) * len(train_data_scaled[0]))
        test_score = model.log_likelihood(test_data_scaled)
        test_av_score = test_score / (len(test_data_scaled) * len(test_data_scaled[0]))
        
        print('training LL: {:.2f}      testing LL: {:.2f}'.format(train_av_score, test_av_score))
        print('')
        print('-----------------------------------------------')

        # store
        hmm_results[n_p, 0, f] = train_score
        hmm_results[n_p, 1, f] = train_av_score
        hmm_results[n_p, 2, f] = test_score
        hmm_results[n_p, 3, f] = test_av_score
        
    for n_p, p_set in enumerate(STICKY_ARHMM_PARAMS):
        obs, k, lag, trans, kappa = [*p_set]
        model = ssm.HMM(k, num_features, observations=obs, observation_kwargs={'lags':lag}, transition=trans, transition_kwargs={'kappa':kappa})
        print('fitting sticky {:} model with k={:} and lags={:}'.format(obs, k, lag))

        # fit
        # shuffle data before fitting
        ridx = np.random.choice(np.arange(len(train_data_scaled)), size=len(train_data_scaled), replace=False)
        train_data_shuffled = [train_data_scaled[i] for i in ridx]
        # model fit
        lls = model.fit(train_data_shuffled, method='em', num_iters=50)

        # record scores
        train_score = model.log_likelihood(train_data_scaled)
        train_av_score = train_score / (len(train_data_scaled) * len(train_data_scaled[0]))
        test_score = model.log_likelihood(test_data_scaled)
        test_av_score = test_score / (len(test_data_scaled) * len(test_data_scaled[0]))
        
        print('training LL: {:.2f}      testing LL: {:.2f}'.format(train_av_score, test_av_score))
        print('')
        print('-----------------------------------------------')

        # store
        shmm_results[n_p, 0, f] = train_score
        shmm_results[n_p, 1, f] = train_av_score
        shmm_results[n_p, 2, f] = test_score
        shmm_results[n_p, 3, f] = test_av_score    
        
    for n_p, p_set in enumerate(GMM_PARAMS):
        obs, k = [*p_set]
        
        concat_data = np.concatenate(train_data_shuffled)
        
        model = GaussianMixture(n_components=k, covariance_type=obs, max_iter=50)
        model.fit(concat_data)

        train_av_score = model.score(concat_data)
        train_score = train_av_score * len(concat_data)
        test_av_score = model.score(np.concatenate(test_data_scaled))
        test_score = test_score * len(np.concatenate(test_data_scaled))
        
        print('fitting gaussian mixture model with k={:}'.format(k))
        print('training LL: {:.2f}      testing LL: {:.2f}'.format(train_av_score, test_av_score))
        print('')
        print('-----------------------------------------------')

        # store
        gmm_results[n_p, 0, f] = train_score
        gmm_results[n_p, 1, f] = train_av_score
        gmm_results[n_p, 2, f] = test_score
        gmm_results[n_p, 3, f] = test_av_score




FOLD 1----------------------------------------------
fitting diagonal_gaussian model with k=1


  0%|          | 0/50 [00:00<?, ?it/s]

training LL: -53.92      testing LL: -53.85

-----------------------------------------------
fitting diagonal_gaussian model with k=2


  0%|          | 0/50 [00:00<?, ?it/s]

training LL: -53.09      testing LL: -53.03

-----------------------------------------------
fitting diagonal_gaussian model with k=5


...

training LL: 93.50      testing LL: 91.76

-----------------------------------------------
fitting diagonal_robust_ar model with k=5 and lags=2


  0%|          | 0/50 [00:00<?, ?it/s]

training LL: 195.07      testing LL: 193.03

-----------------------------------------------
fitting diagonal_robust_ar model with k=5 and lags=5


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Concatenate and save out CV results

In [53]:
# HMM
param_df = pd.DataFrame(ALL_HMM_PARAMS)
param_df.fillna(value=0, inplace=True)

hmm_results_df = pd.DataFrame()
for f in np.arange(CV):
    fold_df = pd.DataFrame(pd.concat((param_df, pd.DataFrame(hmm_results[:,:,f])), axis=1))
    fold_df.columns = ['obs_model', 'states', 'lag', 'train_LL', 'train_av_LL', 'test_LL', 'test_av_LL']
    fold_df.insert(2, 'fold', [f+1]*len(ALL_HMM_PARAMS))
    hmm_results_df = pd.concat((hmm_results_df, fold_df)).reset_index(drop=True)

# STICKY HMM    
param_df = pd.DataFrame(STICKY_ARHMM_PARAMS)
param_df.fillna(value=0, inplace=True)

shmm_results_df = pd.DataFrame()
for f in np.arange(CV):
    fold_df = pd.DataFrame(pd.concat((param_df, pd.DataFrame(shmm_results[:,:,f])), axis=1))
    fold_df.columns = ['obs_model', 'states', 'lag', 'transition', 'kappa', 'train_LL', 'train_av_LL', 'test_LL', 'test_av_LL']
    fold_df.insert(2, 'fold', [f+1]*len(STICKY_ARHMM_PARAMS))
    shmm_results_df = pd.concat((shmm_results_df, fold_df)).reset_index(drop=True)


# GMM    
param_df = pd.DataFrame(GMM_PARAMS)
param_df.fillna(value=0, inplace=True)

gmm_results_df = pd.DataFrame()
for f in np.arange(CV):
    fold_df = pd.DataFrame(pd.concat((param_df, pd.DataFrame(gmm_results[:,:,f])), axis=1))
    fold_df.columns = ['obs_model', 'states', 'train_LL', 'train_av_LL', 'test_LL', 'test_av_LL']
    fold_df.insert(2, 'fold', [f+1]*len(GMM_PARAMS))
    gmm_results_df = pd.concat((gmm_results_df, fold_df)).reset_index(drop=True)
    
    
all_results = pd.concat((hmm_results_df, shmm_results_df, gmm_results_df), axis=0, ignore_index=True)
all_results.to_csv('outputs/cross-validation-results.csv', index=None)